In [0]:
import requests
from IPython.core.display import HTML
HTML(f"""
<style>
@import "https://cdn.jsdelivr.net/npm/bulma@0.9.4/css/bulma.min.css";
</style>
""")

# Gradient Descent / Ascent
This notebook is for illustrating the plain vanilla gradient descent algorithm. In practice you would likely use libraries with more sophisticated implementations such as pytorch or scikitlearn.
## Gradient Ascent on a function
This example shows a basic implementation of the gradient ascent algorithm.


In [0]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
plt.style.use('bmh')
from IPython.core.display import HTML as Center
import itertools as itr

Center(""" <style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style> """)

Define a bivariate (two-variables) function, $f(x,y)=\exp(-(x-2)^2 - (y+1)^2)$, to **maximize** . The true **maximum** is easily seen to be at $x=2$ and $y=-1$.


In [0]:
def f(x, y):
    return np.exp(-(x-2)**2 - (y+1)**2)

Let's make a surface plot to  get a feel for the function.


In [0]:
# The ranges of x and y values that we'd like to include in the contour plot.
x_range = np.linspace(-1, 5, 40)
y_range = np.linspace(-3, 3, 40)

# Form all possible x-y pairs in these ranges.
X, Y = np.meshgrid(x_range, y_range)

# Make the surface plot.
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.plot_surface(X, Y, f(X, Y), cmap='coolwarm')

ax.set_xlabel('x')
ax.set_ylabel('y')
#ax.set_zlabel('f(x, y)')
ax.set_title(r'$f(x,y)=\exp(-(x-2)^2 - (y+1)^2)$');
plt.show()

Let's also make a _contour plot_ that shows the surface of the function $f$. (Dark rings = lower values, light rings = higher values.)


In [0]:
plt.contour(X, Y, f(X, Y), 20, cmap='coolwarm');
plt.xlabel('x')
plt.ylabel('y');

The gradient $\nabla f$ is obtained by differentiating $f$ with respect to the inputs $x$ and $y$


In [0]:
def gradient_of_f(x, y):
    return (-2*(x-2)*f(x, y), -2*(y+1)*f(x, y))

This gradient **ascent** algorithm (`gradient_ascent`
)  seeks the values of $x$ and $y$ that give the **maximum** of $f(x, y)$.
The inputs to `gradient_ascent`
 are the initial value of $x$ and $y$; a threshold that defines  the termination criterion for how small the gradient should be at the maximum; and a learning rate.
The function returns the $x$ and $y$ values that maximize the function. For pedagogical purposes,  the function also returns a list called `history`
 of all the $x$-$y$ coordinates that were eastimated during the optimization procedure.


In [0]:
def gradient_ascent(x_init, y_init, 
                    threshold = 0.001,
                    learningRate = 0.6):
    x = x_init
    y = y_init
    history = [(x, y)]
    done = False
    while not done:        
        gxy = gradient_of_f(x, y)
        x += learningRate * gxy[0]
        y += learningRate * gxy[1]
        history.append([x, y])
        if np.linalg.norm(gxy) < threshold:
            done = True
    return (x, y), history

As shown below, the algorithm gets fairly close to the global optimal value at $x$ and $y$ .


In [0]:
(x, y), history = gradient_ascent(0, 0)

x, y

The contour plot including the intermediate estimates of  (red points) of the gradient ascent algorithm as it climbs from (0, 0) towards the top (2, -1).


In [0]:
# Make the contour plot.
plt.contour(X, Y, f(X, Y), 20, cmap='coolwarm')

# Plot the gradient ascent path.
plt.plot([x for x, _ in history], [y for _, y in history], 'r.');

## Minimzing a Linear Least Squares using Gradient Descent
This next example will show the **minimization** of a loss function using gradient descent.
The objective function is the standard linear least squares:

$$
\text{Loss}(w) = \frac{1}{N} \sum_{i=1}^N (\theta^\top x - y)^2
$$

Linear least squares regression has an _analytical_ (exact) solution that, as discussed ealier in the course,  can be found using the projection / pseudoinverse using the designmatrix $X$:

$$
\theta = (X^T\cdot X)^{-1} \cdot X^T \cdot y
$$
or SVD. 
However this example will use gradient descent for linear regression to exemplify
- gradient descent on a simple and pedagogical example
- that when the design matrix $X$ is very large it may become computationally intractable to compute the full solution using SVD/pseudo inverse. In fact this is the  reason that some of scikit-learn's  linear regression training algorithms (e.g. [`Ridge`
](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html)
, [`Lasso`
](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html)
 etc) are based on various iterative optimization rather than using the exact solution.



In [0]:
class LeastSquaresRegressorGD():

    def __init__(self, n_iter=20, tolerance=1e-5, eta=0.1):
        self.n_iter = n_iter
        self.tolerance = tolerance
        self.eta = eta
    def predict(self, X):
        return X.dot(self.theta)  
    
    def fit(self, X, Y):
        n_instances, n_features = X.shape
        print("N-Features: %d" % n_features)
        self.theta = np.zeros(n_features)
        print("Initial Theta %d" % self.theta)
        self.history = []
        self.thetas = []
        self.thetas.append(self.theta[0])
        for i in range(self.n_iter):            
           
            # predictions on the whole training set using the current estimate theta
            # this is a vector of predicted outputs
            G = X.dot(self.theta)
            
            # vector of error values
            Error = G - Y

            # the loss value for the whole of the training set is the mean of the squared errors
            total_loss = np.mean(Error**2)

            # and the gradient is also computed over the whole training set, and the mean over the
            # individual gradients computed.
            gradient_theta = np.mean(2*Error*X.T, axis=1)

            # if the gradient vector is small enough, terminate early
            if np.linalg.norm(gradient_theta) <= self.tolerance:
                break
            
            # update the weights in the direction of reducing the error for the whole training set. 
            self.theta -= self.eta*gradient_theta
            self.thetas.append(self.theta[0])
            self.history.append(total_loss)
            
        print('GD final loss (iteration {}): {:.4f}'.format(i+1, total_loss))

### Testing the training algorithm on a synthetic dataset
The example uses univariate data linear (one input feature - the slope) model with gaussian noise. 


In [0]:
np.random.seed(0)
numSamples = 1000
slope = -0.9
intercept = 0.4
testsetSize= 400
Xtrain = np.random.normal(size = (numSamples, 1))
Ytrain = slope * Xtrain[:,0] + intercept * np.random.normal(size=Xtrain.shape[0])

Xtest = np.sort(np.random.normal(size = (testsetSize, 1)))
Ytest = slope * Xtest[:,0] + intercept * np.random.normal(size=Xtest.shape[0])

plt.plot(Xtrain, Ytrain, 'g.');
plt.plot(Xtest, Ytest, 'b.');

plt.xlabel("Input feature")
plt.ylabel("Predicted output")
plt.title("Synthetic dataset")

You can play around with how the learning rate affects the convergence of the algorithm. If it's too low, convergence will be slow. If it's too high, convergence may also be slow (because the algorithm "jumps around") or it may not converge at all.


In [0]:
gd_regression = LeastSquaresRegressorGD(n_iter=20, eta=0.25, tolerance=0.005)
gd_regression.fit(Xtrain, Ytrain)

The learning rate above is purposefully (eta) set quite low so that you below can see how the iterations will improve the fit. Setting the value to around $0.$ will have much faster convergence
Plotting the `history`
  during training reveals the progress of the optimization 
The gradient descent algorithm performs reasonably well (but poorly compared to more sophisticated models). That is unsurprising since the algortihm  has not made any updates by that point.


In [0]:
plt.plot(gd_regression.history, '.-')

The evaluation of the MSE on the test set reveals that the model found by the training algorithm is reasonably good.


In [0]:
print(mean_squared_error(Ytest, gd_regression.predict(Xtest)))

In [0]:
for i, theta in enumerate(gd_regression.thetas):
    # Show only the first 10 figures due
    if(i>10):
        break
    gd_regression.theta = theta
    y1 =  gd_regression.predict(np.array(-3))
    y2 =  gd_regression.predict(np.array(3))
    
    plt.figure()

    plt.plot(Xtrain, Ytrain, 'g.');
    plt.plot(Xtest, Ytest, 'b.');

    plt.xlabel("Input feature")
    plt.ylabel("Predicted output")
    plt.title(r"Synthetic dataset (itr = %d, $\theta=$ %2.4f)" % (i,gd_regression.theta))
    
        
    plt.plot([-3,3],[y1,y2],'r-')